In [17]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
%pip install requests


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import ipywidgets as widgets
from IPython.display import HTML, display

import datetime
import difflib
import json
import re
import requests
import sys
import time
from typing import Dict, List, NamedTuple, Optional, Union

import collections
print("imports successfull")

imports successfull


### PersonifiedScriptWriter hyperparameters

In [20]:

DEFAULT_SEED =  1
SAMPLING_PROB =  0.9
SAMPLING_TEMP =  0.9
#Max length for the generated title, place description and other, in tokens
SAMPLE_LENGTH_TITLE = 64 #parameter values [64, 128, 256, 511]
SAMPLE_LENGTH_PLACE = 128 #parameter values [128, 256, 511]
SAMPLE_LENGTH = 511 #parameter values [128, 256, 511]
#Max lengths during repeated sampling, in case `<end>` is not found (default: 2048)
MAX_PARAGRAPH_LENGTH_CHARACTERS = 1024 #parameter values [511, 1024, 2048, 4096]
MAX_PARAGRAPH_LENGTH_SCENES = 1024 #parameter values [511, 1024, 2048, 4096]
MAX_PARAGRAPH_LENGTH = 1024 #parameter values [511, 1024, 2048, 4096]
#max number of retries for the model API before giving up
MAX_RETRIES = 10
#Loop detection: max number of repetitions before resampling, and number of attempts to get out of the loop so that the model dosent generate continous same response in a loop
MAX_NUM_REPETITIONS = 3
MAX_NUM_ATTEMPTS_GET_OUT_OF_LOOP = 3
LLM_API_GENERATE_URL = "http://localhost:5001/api/v1/generate" #kobold running



In [21]:





# Script markers


END_MARKER = '<end>'
STOP_MARKER = '<stop>'
CHARACTER_MARKER = '<character>'
DESCRIPTION_MARKER = '<description>'
SCENES_MARKER = '<scenes>'
DIALOG_MARKER = '<dialog>'
TITLE_ELEMENT = 'Title: '
CHARACTERS_ELEMENT = 'Characters: '
DESCRIPTION_ELEMENT = 'Description: '
PLACE_ELEMENT = 'Place: '
PLOT_ELEMENT = 'Plot element: '
PREVIOUS_ELEMENT = 'Previous beat: '
SUMMARY_ELEMENT = 'Summary: '
BEAT_ELEMENT = 'Beat: '


# Personified-script-Writer entities



class Title(NamedTuple):
  """Title class."""

  title: str

  @classmethod
  def from_string(cls, text: str):
    title = extract_elements(text, TITLE_ELEMENT, END_MARKER)[0]
    return cls(title)

  def to_string(self):
    s = ''
    s += TITLE_ELEMENT + self.title
    s += END_MARKER
    return s


def get_title(title: Title) -> str:
  return title.title


class Character(NamedTuple):
  """Character class."""

  # Name of the character.
  name: str

  # A single sentence describing the character.
  description: str

  @classmethod
  def from_string(cls, text: str):
    elements = text.split(DESCRIPTION_MARKER)
    if len(elements) == 2:
      name = elements[0].strip()
      description = elements[1].strip()
      return cls(name, description)
    else:
      return None


def get_character_description(character: Character) -> str:
  return character.description


class Characters(NamedTuple):
  """Characters class, containing main characters and their descriptions."""

  # A dictionary of character descriptions.
  character_descriptions: Dict[str, str]

  @classmethod
  def from_string(cls, text: str):
    """Parses the characters from the generated text."""
    text = text.strip()

    # Extracts the character descriptions.
    character_descriptions = {}
    elements = extract_elements(text, CHARACTER_MARKER, STOP_MARKER)
    for text_character in elements:
      character = Character.from_string(text_character)
      if character is not None:
        character_descriptions[character.name] = character.description
    return cls(character_descriptions)

  def to_string(self):
    s = '\n'
    for name, description in self.character_descriptions.items():
      s += '\n' + CHARACTER_MARKER + ' ' + name + ' ' + DESCRIPTION_MARKER + ' '
      s += description + ' ' + STOP_MARKER + '\n'
    s += END_MARKER
    return s


def get_character_descriptions(characters: Characters) -> Dict[str, str]:
  return characters.character_descriptions


#class Scene to be present ----


#class Scenes to be present ----


#class place to be present -------


class Story(NamedTuple):
  """Story class."""

  # A storyline is a single sentence summary of the whole plot.
  storyline: str

  # A title for the story.
  title: str

  # Map from character names to full descriptions.
  character_descriptions: Dict[str, str]


  # List of dialogs, one for each scene.
  dialogs: List[str]


def extract_elements(text: str, begin: str, end: str) -> List[str]:
  """Extracts elements from a text string given string and ending markers."""

  results = []
  start = 0
  while True:
    start = text.find(begin, start)
    if start == -1:
      return results
    finish = text.find(end, start)
    if finish == -1:
      return results
    results.append(text[start + len(begin):finish].strip())
    start = finish + len(end)


def strip_remove_end(text: str) -> str:
  text = text.strip()
  end_marker_stripped = END_MARKER.strip()
  if text.endswith(end_marker_stripped):
    text = text[:-len(end_marker_stripped)]
  return text



# Rendering of generated stories











In [22]:

# Language API definition


_MAX_RETRIES = 10
_TIMEOUT = 120.0


class LanguageResponse(NamedTuple):
  prompt: str
  prompt_length: int
  text: str
  text_length: int


class LanguageAPI:
  """Language model wrapper."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):

    self._sample_length = sample_length
    self._model = model
    self._model_param = model_param
    self._config_sampling = config_sampling
    self._seed = seed
    self._max_retries = max_retries
    self._timeout = timeout

  @property
  def default_sample_length(self):
    return self._sample_length

  @property
  def model(self):
    return self._model

  @property
  def model_param(self):
    return self._model_param

  @property
  def model_metadata(self):
    return None

  @property
  def seed(self):
    return self._seed

  @property
  def config_sampling(self):
    return self._config_sampling

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt, optional sample_length and seed."""
    raise NotImplementedError('sample method not implemented in generic class')


class FilterAPI:
  """Filter model wrapper."""

  def validate(self, text: str):
    raise NotImplementedError('validate not implemented in generic class')



# Personifiedscript wtiter Generator



def generate_text(generation_prompt: str,
                  client: LanguageAPI,
                  filter: Optional[FilterAPI] = None,
                  sample_length: Optional[int] = None,
                  max_paragraph_length: int = MAX_PARAGRAPH_LENGTH,
                  seed: Optional[int] = None,
                  num_samples: int = 1,
                  max_num_repetitions: Optional[int] = None) -> str:
  """Generate text using the generation prompt."""

  # To prevent lengthy generation loops, we cap the number of calls to the API.
  if sample_length is None:
    sample_length = client.default_sample_length
  max_num_calls = int(max_paragraph_length / sample_length) + 1
  num_calls = 0

  result = ''
  while True:
    prompt = generation_prompt + result
    success, current_seed = False, seed
    while success is False:
      t0 = time.time()
      responses = client.sample(
          prompt=prompt,
          sample_length=sample_length,
          seed=current_seed,
          num_samples=num_samples)
      t1 = time.time()
      # Get the first result from the list of responses
      response = responses[0]
      if filter is not None and not filter.validate(response.text):
        return 'Content was filtered out.' + END_MARKER
      if max_num_repetitions:
        success = not detect_loop(
            response.text, max_num_repetitions=max_num_repetitions)
        if not success:
          current_seed += 1
          if current_seed > (seed + MAX_NUM_ATTEMPTS_GET_OUT_OF_LOOP):
            success = True
          else:
            continue
      else:
        success = True

    result = result + response.text
    num_calls += 1

    # Attempt to find the END_MARKER
    index = result.find(END_MARKER)
    if index != -1:
      return result[:index] + END_MARKER

    # Attempt to find the start of a new example
    index = result.find('Example ')
    if index != -1:
      return result[:index] + END_MARKER

    if max_paragraph_length is not None and len(result) > max_paragraph_length:
      return result + END_MARKER
    if num_calls >= max_num_calls:
      return result + END_MARKER

  return result


def generate_text_no_loop(generation_prompt: str,
                          client: LanguageAPI,
                          filter: Optional[FilterAPI] = None,
                          sample_length: Optional[int] = None,
                          max_paragraph_length: int = MAX_PARAGRAPH_LENGTH,
                          seed: Optional[int] = None,
                          num_samples: int = 1) -> str:
  """Generate text using the generation prompt, without any loop."""
  return generate_text(
      generation_prompt=generation_prompt,
      client=client,
      filter=filter,
      sample_length=sample_length,
      max_paragraph_length=sample_length,
      seed=seed,
      max_num_repetitions=None,
      num_samples=num_samples)


def generate_title(storyline: str,
                   prefixes: Dict[str, str],
                   client: LanguageAPI,
                   filter: Optional[FilterAPI] = None,
                   seed: Optional[int] = None,
                   num_samples: int = 1):
  """Generate a title given a storyline, and client."""

  # Combine the prompt and storyline as a helpful generation prefix
  titles_prefix = prefixes['TITLES_PROMPT'] + storyline + ' ' + TITLE_ELEMENT
  title_text = generate_text_no_loop(
      generation_prompt=titles_prefix,
      client=client,
      filter=filter,
      sample_length=SAMPLE_LENGTH_TITLE,
      seed=seed,
      num_samples=num_samples)
  title = Title.from_string(TITLE_ELEMENT + title_text)
  return (title, titles_prefix)


def generate_characters(
    storyline: str,
    prefixes: Dict[str, str],
    client: LanguageAPI,
    filter: Optional[FilterAPI] = None,
    seed: Optional[int] = None,
    max_paragraph_length: int = (MAX_PARAGRAPH_LENGTH_CHARACTERS),
    num_samples: int = 1):
  """Generate characters given a storyline, prompt, and client."""

  # Combine the prompt and storyline as a helpful generation prefix
  characters_prefix = prefixes['CHARACTERS_PROMPT'] + storyline
  characters_text = generate_text(
      generation_prompt=characters_prefix,
      client=client,
      filter=filter,
      seed=seed,
      max_paragraph_length=max_paragraph_length,
      num_samples=num_samples)
  characters = Characters.from_string(characters_text)

  return (characters, characters_prefix)











def detect_loop(text: str, max_num_repetitions: int = MAX_NUM_REPETITIONS):
  """Detect loops in generated text."""

  blocks = text.split('\n\n')
  num_unique_blocks = collections.Counter(blocks)
  for block in blocks:
    num_repetitions = num_unique_blocks[block]
    if num_repetitions > max_num_repetitions:
      print(f'Detected {num_repetitions} repetitions of block:\n{block}')
      return True
  return False





def diff_prompt_change_str(prompt_before: str, prompt_after: str) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # For the current element, compare prompts line by line.
  res = difflib.unified_diff(
      prompt_before.split('\n'), prompt_after.split('\n'))
  diff = ''
  for line in res:
    line = line.strip()
    if line != '---' and line != '+++' and not line.startswith('@@'):
      if len(line) > 1 and (line.startswith('+') or line.startswith('-')):
        diff += line + '\n'
  if diff.endswith('\n'):
    diff = diff[:-1]
  return diff


def diff_prompt_change_list(prompt_before: List[str],
                            prompt_after: List[str]) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # Handle deletions and insertions.
  len_before = len(prompt_before)
  len_after = len(prompt_after)
  if len_before > len_after:
    return 'Deleted element'
  if len_before < len_after:
    return 'Added new element'

  diffs = [
      diff_prompt_change_str(a, b)
      for (a, b) in zip(prompt_before, prompt_after)
  ]
  return '\n'.join([diff for diff in diffs if len(diff) > 0])





def diff_prompt_change_dict(prompt_before: Dict[str, str],
                            prompt_after: Dict[str, str]) -> str:
  """Return a text diff on prompt sets `prompt_before` and `prompt_after`."""

  # Loop over the keys in the prompts to compare them one by one.
  keys_before = sorted(prompt_before.keys())
  keys_after = sorted(prompt_after.keys())
  diffs = [
      diff_prompt_change_str(a, b) for (a, b) in zip(keys_before, keys_after)
  ]
  diff_keys = '\n'.join([diff for diff in diffs if len(diff) > 0])
  # Loop over the values in the prompts to compare them one by one.
  values_before = sorted(prompt_before.values())
  values_after = sorted(prompt_after.values())
  diffs = [
      diff_prompt_change_str(a, b)
      for (a, b) in zip(values_before, values_after)
  ]
  diff_values = '\n'.join([diff for diff in diffs if len(diff) > 0])
  return diff_keys + diff_values


class StoryGenerator:
  """Generate a story from the provided storyline, using the client provided."""

  level_names = ('storyline', 'title', 'characters', 'scenes', 'places',
                 'dialogs')

  def __init__(
      self,
      storyline: str,
      prefixes: Dict[str, str],
      max_paragraph_length: int = 1024,
      max_paragraph_length_characters: int = (MAX_PARAGRAPH_LENGTH_CHARACTERS),
      max_paragraph_length_scenes: int = (MAX_PARAGRAPH_LENGTH_SCENES),
      num_samples: int = 1,
      client: Optional[LanguageAPI] = None,
      filter: Optional[FilterAPI] = None):
    self._prefixes = prefixes
    self._max_paragraph_length = max_paragraph_length
    self._max_paragraph_length_characters = max_paragraph_length_characters
    self._max_paragraph_length_scenes = max_paragraph_length_scenes
    self._num_samples = num_samples
    self._client = client
    self._filter = filter

    # Prompts and outputs of the hierarchical generator are organised in levels.
    self.prompts = {
        'title': '',
        'characters': '',
        'scenes': '',
        'places': {
            '': ''
        },
        'dialogs': ['']
    }
    self._title = Title('')
    self._characters = Characters({'': ''})
    

    # History of interventions.
    self.interventions = {}
    self._set_storyline(storyline)

  def _set_storyline(self, storyline: str):
    """Set storyline and initialise the outputs of the generator."""
    self._level = 0

    # Add period to the end of the storyline, unless there is already one there.
    if storyline.find('.') == -1:
      storyline = storyline + '.'
    self._storyline = storyline

    # Keep track of each storyline intervention.
    timestamp = time.time()
    self.interventions[timestamp] = 'STORYLINE\n' + storyline

  @property
  def seed(self):
    return self._client.seed

  @property
  def title(self) -> Title:
    """Return the title."""
    return self._title

  @property
  def characters(self) -> Characters:
    """Return the characters."""
    return self._characters

  

  

  def title_str(self) -> str:
    """Return the title as a string."""
    return self._title.title

  

  def step(self,
           level: Optional[int] = None,
           seed: Optional[int] = None,
           idx: Optional[int] = None) -> bool:
    """Step down a level in the hierarchical generation of a story."""

    # Move to the next level of hierarchical generation.
    if level is None:
      level = self._level
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    level += 1
    self._level = level

    # Keep track of each step intervention.
    timestamp = time.time()
    self.interventions[timestamp] = 'STEP ' + str(level) + '\n'

    if level == 1:
      # Step 1: Generate title given a storyline.
      (title, titles_prefix) = generate_title(
          storyline=self._storyline,
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          seed=seed)
      self._title = title
      self.prompts['title'] = titles_prefix
      self.interventions[timestamp] += title.to_string()
      success = len(title.title) > 0
      return success

    if level == 2:
      # Step 2: Generate characters given a storyline.
      (characters, character_prompts) = generate_characters(
          storyline=self._storyline,
          prefixes=self._prefixes,
          client=self._client,
          filter=self._filter,
          num_samples=self._num_samples,
          max_paragraph_length=self._max_paragraph_length_characters,
          seed=seed)
      self._characters = characters
      self.prompts['characters'] = character_prompts
      self.interventions[timestamp] += characters.to_string()
      success = len(characters.character_descriptions) > 0
      return success









  def rewrite(self, text, level=0, entity=None):
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    prompt_diff = None

    if level == 0:
      # Step 0: Rewrite the storyline and begin new story.
      prompt_diff = diff_prompt_change_str(self._storyline, text)
      self._set_storyline(text)

    if level == 1:
      # Step 1: Rewrite the title.
      title = Title.from_string(text)
      prompt_diff = diff_prompt_change_str(self._title.title, title.title)
      self._title = title

    if level == 2:
      # Step 2: Rewrite the characters.
      characters = Characters.from_string(text)
      prompt_diff = diff_prompt_change_dict(
          self._characters.character_descriptions,
          characters.character_descriptions)
      self._characters = characters






    # Keep track of each rewrite intervention.
    if prompt_diff is not None and len(prompt_diff) > 0:
      timestamp = time.time()
      self.interventions[timestamp] = 'REWRITE ' + self.level_names[level]
      if entity:
        self.interventions[timestamp] += ' ' + str(entity)
      self.interventions[timestamp] += prompt_diff

  def complete(self,
               level=0,
               seed=None,
               entity=None,
               sample_length=SAMPLE_LENGTH):
    if level < 0 or level >= len(self.level_names):
      raise ValueError('Invalid level encountered on step.')
    prompt_diff = None

    if level == 2:
      # Step 2: Complete the characters.
      text_characters = self._characters.to_string()
      text_characters = strip_remove_end(text_characters)
      prompt = self.prompts['characters'] + text_characters
      text = generate_text(
          generation_prompt=prompt,
          client=self._client,
          filter=self._filter,
          sample_length=sample_length,
          max_paragraph_length=sample_length,
          seed=seed,
          num_samples=1)
      new_characters = Characters.from_string(text_characters + text)
      prompt_diff = diff_prompt_change_dict(
          self._characters.character_descriptions,
          new_characters.character_descriptions)
      self._characters = new_characters





    # Keep track of each rewrite intervention.
    if prompt_diff is not None and len(prompt_diff) > 0:
      timestamp = time.time()
      self.interventions[timestamp] = 'COMPLETE ' + self.level_names[level]
      if entity:
        self.interventions[timestamp] += ' ' + str(entity)
      self.interventions[timestamp] += prompt_diff




In [23]:

# UI



class GenerationAction:
  NEW = 1
  CONTINUE = 2
  REWRITE = 3


class GenerationHistory:
  """Custom data structure to handle the history of GenerationAction edits:

  NEW, CONTINUE or REWRITE. Consecutive REWRITE edits do not add to history.
  """

  def __init__(self):
    self._items = []
    self._actions = []
    self._idx = -1
    self._locked = False

  def _plain_add(self, item, action: GenerationAction):
    self._items.append(item)
    self._actions.append(action)
    self._idx = len(self._items) - 1
    return self._idx

  def add(self, item, action: GenerationAction):
    if len(self._items) == 0 or action != GenerationAction.REWRITE:
      return self._plain_add(item, action)
    last_action = self._actions[-1]
    if last_action != GenerationAction.REWRITE:
      return self._plain_add(item, action)
    self._items[self._idx] = item
    return self._idx

  def previous(self):
    if len(self._items) == 0:
      return None
    self._idx = max(self._idx - 1, 0)
    return self._items[self._idx]

  def next(self):
    if len(self._items) == 0:
      return None
    self._idx = min(self._idx + 1, len(self._items) - 1)
    return self._items[self._idx]

filter = None

print('PersonifiedScriptWriter set-up complete.')

PersonifiedScriptWriter set-up complete.


### KOBOLDCPP API CONF FOR LOCAL MODEL CONN

In [24]:

import requests
llm_base_url = LLM_API_GENERATE_URL


class Kobold(LanguageAPI):
  """A class wrapping the GPT-3 model API from OpenAI."""

  def __init__(self,
               sample_length: int,
               model: Optional[str] = None,
               model_param: Optional[str] = None,
               config_sampling: Optional[dict] = None,
               seed: Optional[int] = None,
               max_retries: int = _MAX_RETRIES,
               timeout: float = _TIMEOUT):
    """Initializer.

    Args:
      sample_length: Length of text to sample from model.
      model: The model name to correct to. An error will be raised if it does
        not exist.
      model_param: GPT-3 API key.
      config_sampling: ConfigDict with parameters.
      seed: Random seed for sampling.
      max_retries: Maximum number of retries for the remote API.
      timeout: Maximum waiting timeout
    """
    super().__init__(sample_length=sample_length,
                     model=model,
                     model_param=model_param,
                     config_sampling=config_sampling,
                     seed=seed,
                     max_retries=max_retries,
                     timeout=timeout)



  @property
  def client(self):
    return self._client

  @property
  def model_metadata(self):
    return {'engine': self._model,
            'api_key': self._model_param,
            'max_tokens': self._sample_length}

  def sample(self,
             prompt: str,
             sample_length: Optional[int] = None,
             seed: Optional[int] = None,
             num_samples: int = 1):
    """Sample model with provided prompt and optional sample_length and seed."""
    if sample_length is None:
      sample_length = self._sample_length
    if seed is None:
      seed = self._seed
    history = []
    history.append({"role": "user", "content": prompt})      
    """ "mode": "chat","character": "Example" """
    
    
    data = {
        "prompt": prompt,
#        "max_context_length": 2048,
        "max_length": sample_length,
        "temperature": self._config_sampling['temp'],
        "top_p": self._config_sampling['prob'],
        "rep_pen": 1.1,
#        "sampler_seed": 10,
    }
    headers = {
        "Content-Type": "application/json"
    }
    #print(llm_base_url)
    #print(data)
    #print(self._max_retries)
      
    for attempt in range(self._max_retries):
      try:
        response = requests.post(llm_base_url, headers=headers, json=data, verify=False)
        results = []
        result_llm = response.json()
        for k in range(len(result_llm["results"])):
          results.append(LanguageResponse(
                  text=result_llm["results"][k]["text"],
                  text_length=len(result_llm["results"][k]["text"]),
                  prompt=prompt,
                  prompt_length=512))
        return results
      except Exception as ex:
        print(f'Attempt {attempt + 1} of {self._max_retries}: API call failed.')
        last_exception = ex
    raise last_exception

# Create the config. - this is from the GPT example, most of these are unused and can be removed
config = {}
config['language_api_name'] = 'kobold'
config['model_api_key'] = 'api'
config['model_name'] = 'modelname'
config['max_retries'] = MAX_RETRIES
config['sample_length'] = SAMPLE_LENGTH
config['max_paragraph_length'] = MAX_PARAGRAPH_LENGTH
config['max_paragraph_length_characters'] = MAX_PARAGRAPH_LENGTH_CHARACTERS
config['max_paragraph_length_scenes'] = MAX_PARAGRAPH_LENGTH_SCENES
config['sampling'] = {}
config['sampling']['prob'] = SAMPLING_PROB
config['sampling']['temp'] = SAMPLING_TEMP
config['sampling']['frequency_penalty'] = 1.23
config['sampling']['presence_penalty'] = 0.9
config['prefixes'] = {}
config['file_dir'] = None

print('Config:')
for key, value in config.items():
  if key != 'prefixes':
    print(f'{key}: {value}')

client = Kobold(
    model_param=config['model_api_key'],
    model=config['model_name'],
    seed=DEFAULT_SEED,
    sample_length=config['sample_length'],
    max_retries=config['max_retries'],
    config_sampling=config['sampling'])

print(f'Client model metadata: {client.model_metadata}')

Config:
language_api_name: kobold
model_api_key: api
model_name: modelname
max_retries: 10
sample_length: 511
max_paragraph_length: 1024
max_paragraph_length_characters: 1024
max_paragraph_length_scenes: 1024
sampling: {'prob': 0.9, 'temp': 0.9, 'frequency_penalty': 1.23, 'presence_penalty': 0.9}
file_dir: None
Client model metadata: {'engine': 'modelname', 'api_key': 'api', 'max_tokens': 511}


In [25]:
# Simple test to check if the API is hooked up
test_result = client.sample("hello hello mike testing?")
print(test_result)

[LanguageResponse(prompt='hello hello mike testing?', prompt_length=512, text="\n∅ Testing, 1234567890 (4).png origin?testing pngOrigin: MikeHello. Mike Hello! I am Mike, and I am here to help you with any questions or problems you may have. Please let me know how I can assist you today.hello mike testing 😊👨\u200d💻 Is there anything specific you would like to chat about or ask? I'm here to listen and help in any way I can.", text_length=341)]


### prompt-prefix sets

In [26]:
#Medea



#Log line: `Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.`


medea_prefixes = {}
medea_prefixes['CHARACTERS_PROMPT'] = """
Example 1. Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.
""" + CHARACTER_MARKER + """Medea """ + DESCRIPTION_MARKER + """ Medea is the protagonist of the play. A sorceress and a princess, she fled her country and family to live with Jason in Corinth, where they established a family of two children and gained a favorable reputation. Jason has divorced Medea and taken up with a new family.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Jason """ + DESCRIPTION_MARKER + """ Jason is considered the play's villain, though his evil stems more from weakness than strength. A former adventurer, Jason abandons his wife, Medea, in order to marry the beautiful young daughter of Creon, King of Corinth, and fuels Medea to a revenge.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Women of Corinth """ + DESCRIPTION_MARKER + """ The Women of Corinth are a commentator to the action. They fully sympathizes with Medea's plight, excepting her decision to murder her own children.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Creon """ + DESCRIPTION_MARKER + """ Creon is the King of Corinth, banishes Medea from the city.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """The Nurse """ + DESCRIPTION_MARKER + """ The Nurse is the caretaker of the house and of the children and serves as Medea's confidant.""" + STOP_MARKER + """
""" + END_MARKER + """
Example 2. """




medea_prefixes['TITLES_PROMPT'] = """
Examples of alternative, original and descriptive titles for known play and film scripts.

Example 1. Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess of the kingdom of Colchis, and the wife of Jason, finds her position in the Greek world threatened as Jason leaves her for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as her own two sons, after which she escapes to Athens. Title: A Feminist Tale""" + END_MARKER + """
Example 2. Ancient Greek tragedy that deals with Antigone’s burial of her brother Polynices, in defiance of the laws of Creon and the state, and the tragic repercussions of her act of civil disobedience. Title: In My Brother's Name""" + END_MARKER + """
Example 3. Greek comedy that tells the story of the god Dionysus (also known to the Greeks as Bacchus) who, despairing of the current state of Athens’ tragedians, travels to Hades with his slave Xanthias to bring Euripides back from the dead. Title: Dionysus in Hades""" + END_MARKER + """
Example 4. """





In [27]:
#Sci-Fi

# Log line for Star Wars: Episode IV: `A science-fiction fantasy about a naive but ambitious urban boy from a backwater desert who discovers powers he never knew he had when he teams up with a naughty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.`

#

scifi_prefixes = {}
scifi_prefixes['CHARACTERS_PROMPT'] = """
Example 1. A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire.
""" + CHARACTER_MARKER + """Luke Skywalker """ + DESCRIPTION_MARKER + """Luke Skywalker is the hero. A naive farm boy, he will discover special powers under the guidance of mentor Ben Kenobi.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Ben Kenobi """ + DESCRIPTION_MARKER + """Ben Kenobi is the mentor figure. A recluse Jedi warrior, he will take Luke Skywalker as apprentice.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Darth Vader """ + DESCRIPTION_MARKER + """Darth Vader is the antagonist. As a commander of the evil Galactic Empire, he controls space station The Death Star.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Princess Leia """ + DESCRIPTION_MARKER + """Princess Leia is a feisty and brave leader of the Rebellion. She holds the plans of the Death Star. She will become Luke's friend.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Han Solo """ + DESCRIPTION_MARKER + """Han Solo is a brash mercenary space pilot of the Millenium Falcon and a friend of Chebacca. He will take Luke on his spaceship.""" + STOP_MARKER + """
""" + CHARACTER_MARKER + """Chewbacca """ + DESCRIPTION_MARKER + """Chewbacca is a furry and trustful monster. He is a friend of Han Solo and a copilot on the Millemium Falcon.""" + STOP_MARKER + """
""" + END_MARKER + """
Example 2. """



scifi_prefixes['TITLES_PROMPT'] = """
Examples of alternative, original and descriptive titles for known play and film scripts.

Example 1. A science-fiction fantasy about a naive but ambitious farm boy from a backwater desert who discovers powers he never knew he had when he teams up with a feisty princess, a mercenary space pilot and an old wizard warrior to lead a ragtag rebellion against the sinister forces of the evil Galactic Empire. Title: The Death Star's Menace""" + END_MARKER + """
Example 2. Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens. Title: The Day The Earth Was Saved By Outer Space.""" + END_MARKER + """
Example 3. """






In [28]:
#Choosing among prefix sets

prefix_set = 'scifi_prefixes' #parameters ['medea_prefixes', 'scifi_prefixes']
prefixes = eval(prefix_set)
config['prefixes'] = prefixes

print(f'Loaded {prefix_set}.')

Loaded scifi_prefixes.


In [33]:
#Defining the Log line or initial input for the story its like a gist.

#Log lines are one- or two-sentence summaries of the action.
#They normally contain the setting, protagonist, antagonist, a conflict or goal and sometimes the inciting incident.
logline = "Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens."
print(logline)


Ancient Greek tragedy based upon the myth of Jason and Medea. Medea, a former princess and the wife of Jason, finds her position in the Greek world threatened as Jason leaves Medea for a Greek princess of Corinth. Medea takes vengeance on Jason by murdering his new wife as well as Medea's own two sons, after which she escapes to Athens.


In [34]:
#@title Create the story generator

generator = StoryGenerator(
    storyline=logline,
    prefixes=prefixes,
    max_paragraph_length=config['max_paragraph_length'],
    client=client,
    filter=filter)

print(f'New ScriptWriting generator created.')

story = None

New ScriptWriting generator created.


### Generate TITLE FOR USER-LINE

In [35]:
#Generate a **Title**

data_title = {"text": "", "text_area": None, "seed": generator.seed - 1}

def fun_generate_title(button):
  data_title["seed"] += 1
  seed = data_title["seed"]
  new_title_button.description = f"Generating {seed}..."
  generator.step(0, seed=seed)
  data_title["text"] = generator.title_str().strip()
  new_title_button.description = "Generate new"
  if data_title["text_area"] is not None:
    data_title["text_area"].value = data_title["text"]

def fun_rewrite_title(text):
  text_to_parse = TITLE_ELEMENT + text + END_MARKER
  generator.rewrite(text_to_parse, level=1)
  return text

# Widget to generate new title.
new_title_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_title_button.on_click(fun_generate_title)
display(new_title_button)

# Widget to rewrite the title.
layout = widgets.Layout(height='50px', min_height='60px', width='auto')
data_title["text_area"] = widgets.Textarea(
    value=data_title["text"], layout=layout, description=' ',
    style={'description_width': 'initial'})
textarea_title = widgets.interactive(
    fun_rewrite_title, text=data_title["text_area"])
display(textarea_title)
data_title["text_area"].value = data_title["text"]
fun_generate_title(new_title_button)

Button(description='Generate new', icon='check', style=ButtonStyle(), tooltip='Generate new')

interactive(children=(Textarea(value='', description=' ', layout=Layout(height='50px', min_height='60px', widt…

In [36]:
#Generate **Characters**

data_chars = {"text": "", "text_area": None, "seed": generator.seed - 1,
              "history": GenerationHistory(), "lock": False}

def fun_generate_characters(_):
  data_chars["seed"] += 1
  seed = data_chars["seed"]
  data_chars["lock"] = True
  while True:
    new_characters_button.description = f"Generating {seed}..."
    generator.step(1, seed=seed)
    data_chars["text"] = strip_remove_end(generator.characters.to_string())
    # Test if characters were actually generated.
    if len(data_chars["text"]) == 0:
      seed += 1
    else:
      break
  data_chars["seed"] = seed
  data_chars["history"].add(data_chars["text"], GenerationAction.NEW)
  new_characters_button.description = "Generate new"
  if data_chars["text_area"] is not None:
    data_chars["text_area"].value = data_chars["text"]
  data_chars["lock"] = False

def fun_continue_characters(_):
  data_chars["seed"] += 1
  seed = data_chars["seed"]
  data_chars["lock"] = True
  characters_continue_button.description = f"Generating {seed}..."
  generator.complete(level=2, seed=seed, sample_length=256)
  data_chars["text"] = strip_remove_end(generator.characters.to_string())
  data_chars["history"].add(data_chars["text"], GenerationAction.CONTINUE)
  characters_continue_button.description = "Continue generation"
  if data_chars["text_area"] is not None:
    data_chars["text_area"].value = data_chars["text"]
  data_chars["lock"] = False

def fun_back_forward(data, history: GenerationHistory, delta: int):
  data["lock"] = True
  if delta > 0:
    data["text"] = history.next()
  if delta < 0:
    data["text"] = history.previous()
  if data["text"] is not None and data["text_area"] is not None:
      data["text_area"].value = data["text"]
  data["lock"] = False

def fun_back_characters(_):
  fun_back_forward(data_chars, data_chars["history"], -1)

def fun_forward_characters(_):
  fun_back_forward(data_chars, data_chars["history"], 1)

def fun_rewrite_characters(text):
  data_chars["text"] = text
  text_to_parse = text + END_MARKER
  generator.rewrite(text_to_parse, level=2)
  if data_chars["lock"] is False:
    data_chars["history"].add(text, GenerationAction.REWRITE)
  return text

# Widget to generate new characters.
new_characters_button = widgets.Button(button_style='', icon='check',
    description='Generate new', tooltip='Generate new', disabled=False)
new_characters_button.on_click(fun_generate_characters)
# Widget to continue the generation of the current characters.
characters_continue_button = widgets.Button(button_style='', icon='check',
    description='Continue generation', tooltip='Continue generation',
    disabled=False)
characters_continue_button.on_click(fun_continue_characters)
# Widgets to move back and forward in history of generation.
back_characters_button = widgets.Button(button_style='', icon='check',
    description='Previous', tooltip='Previous', disabled=False)
back_characters_button.on_click(fun_back_characters)
forward_characters_button = widgets.Button(button_style='', icon='check',
    description='Next', tooltip='Next', disabled=False)
forward_characters_button.on_click(fun_forward_characters)


display(widgets.HBox([new_characters_button, characters_continue_button,
                      back_characters_button, forward_characters_button]))

layout = widgets.Layout(height='390px', min_height='400px', width='auto')
data_chars["text_area"] = widgets.Textarea(
    value=data_chars["text"], layout=layout, description=' ',
    style={'description_width': 'initial'})
textarea_chars = widgets.interactive(
    fun_rewrite_characters, text=data_chars["text_area"])
display(textarea_chars)

fun_generate_characters(new_characters_button)

interactive(children=(Textarea(value='', description=' ', layout=Layout(height='390px', min_height='400px', wi…